In [1]:
!pip install transformers datasets trl huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00


In [4]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
import evaluate  # To calculate ROUGE metrics

In [5]:
data_path = 'Recommendation.csv'
df = pd.read_csv(data_path)
dataset = Dataset.from_pandas(df)

In [9]:
def process_data_for_dpo(example):
    messages = [{"role": "user", "content": f"Recommend a food list for a person whose gender is {example['Cinsiyet']}, age: {example['Yaş']}, hypertension: {example['Hipertansiyon']}, diabetes: {example['Diyabet']}"},
                {"role": "assistant", "content": ", ".join(eval(example['Recommended Foods']))}]
    return {"messages": messages}

In [10]:

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [11]:

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [12]:
# Veri setlerini işlemden geçirme
processed_train_dataset = train_dataset.map(process_data_for_dpo)
processed_val_dataset = val_dataset.map(process_data_for_dpo)

Map:   0%|          | 0/4944 [00:00<?, ? examples/s]

Map:   0%|          | 0/1236 [00:00<?, ? examples/s]

In [13]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=6dbf60a277cce09b4b10d3e552c62f6a5114e5da30f5386c7a792950f23ec196
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [32]:
# Metric computation
rouge = evaluate.load("rouge")
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions[0]

    # Ensure predictions and labels are matched in size
    pred_ids = pred_ids[:len(labels_ids)]

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str,
        references=label_str,
        rouge_types=["rouge1", "rouge2", "rougeL", "rougeLsum"],
    )

    # Compute Accuracy
    accuracy = sum(p == t for p, t in zip(pred_str, label_str)) / len(label_str)

    return {
        "R1": round(rouge_output["rouge1"], 4),
        "R2": round(rouge_output["rouge2"], 4),
        "RL": round(rouge_output["rougeL"], 4),
        "RLsum": round(rouge_output["rougeLsum"], 4),
        "Accuracy": round(accuracy, 4),
    }


In [33]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Memory optimization: avoids storing unnecessary tensors.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [34]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

model.config.use_cache = False


In [35]:
os.environ["WANDB_DISABLED"] = "true"

# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=1,  # Set according to your GPU memory capacity
    gradient_accumulation_steps=4,
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    # eval_steps=50,  # Frequency of evaluation
    fp16=True,  # Mixed precision
)


trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=processed_train_dataset,
    eval_dataset=processed_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,  # Optimize memory usage
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-35-80741c823781>:18: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/4944 [00:00<?, ? examples/s]

Map:   0%|          | 0/1236 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [36]:
# Train the model
trainer.train()

# Modeli kaydetme
finetune_name = "SmolLM2-FT-MyDataset"
trainer.save_model(f"./{finetune_name}")

Step,Training Loss,Validation Loss,R1,R2,Rl,Rlsum,Accuracy
10,2.230200,0.937266,0.021500,0.000000,0.021500,0.021500,0.000000
20,0.781000,0.594340,0.021200,0.000000,0.021200,0.021200,0.000000


KeyboardInterrupt: 

In [ ]:
prompt = "Recommend a food list for a person whose gender is male, age: 53, hypertension: yes, diabetes: yes"
test_prompt = [{"content": prompt, "role": "user"}]

formatted_prompt = tokenizer.apply_chat_template(test_prompt, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

In [ ]:
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=100)
print("Model Çıktısı:", tokenizer.decode(outputs[0], skip_special_tokens=True))

Model Çıktısı: user
Recommend a food list for a person whose gender is male, age: 53, hypertension: yes, diabetes: yes
assistant
Drumstick tree, Onion rings, Focaccia, Espresso, Pigeon pea, Gewürztraminer, Pigeon pea, Gewürztraminer, Pigeon pea, Gewürztraminer, Pigeon pea, Pigeon pea, Pige
